In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

/Users/jrpowers-luhn/miniconda3/envs/579/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
np.random.seed(3)

In [3]:
%run utilities.py

In [4]:
x, y = load_matlab_data('hwkdataNEW.mat')

In [5]:
xtr, ytr, xts, yts, xv, yv = train_test_val_split(x, y)

In [ ]:
scaler = StandardScaler()
xtr, ytr = scaler.fit_trans